## Example: Twitter API in Python, with OAuth1 authentication

<img src="images/Python_Twitter.jpg" width="300px">

The guide below shows the steps of registering an application that you can integrate with **Twitter**. Registering the application is the first step in developing an application that is integrated into its users' social graph.

## Register a new application

All **Twitter** users are potentially **Twitter** application developers. Simply visit https://apps.twitter.com/ and sign in with your **Twitter** credentials.

Click the "Create a new application" button near the top. A new page with the **_Create an application_** form requires basic information about your application.

<img src="images/create_an_application_1.jpg">

- In the **_Name_** field name your application in 32 characters or fewer. 
- In the **_Description_** field describe your application in 10 to 200 characters.
- In the **_Website_** field, you can give a pointer to your website. (It does not really matter, as we will not be using that functionality yet).
- The **_Callback URL_** leave it empty for now.

The **Developer Agreement** section outlines rules you must agree to follow if you build an application that uses **Twitter’s** API. If you agree to the rules, check "Yes, I agree".

<img src="images/create_an_application_2.jpg">

After Captcha challenge click "Create your **Twitter** application" to complete the form and go to the application settings page.

In the opened window you’ll then be presented with lots of information. The main fields to note are **_Consumer key_** and **_Consumer secret_**. These values are your application’s credentials for **Twitter**. You need them to do almost anything with **Twitter**, including going through the OAuth authorization flow and working with **Twitter’s** REST API. But we’re not quite done yet. We now need to authorise the **Twitter** app for your **Twitter** account. To do this you should select "Key and Access Tokens" menu button and create your access token. This **access token** will allow your **Twitter** application to read **Twitter** information. You’ll be able to get data of your tweets, mentions, lists and more. 

<img src="images/create_an_application_3.jpg">

As a result you will see the window with consumer and token keys for your application 

<img src="images/create_an_application_4.jpg">

Above the four fields are highlighted. You will need these long horrible strings of characters for your **Twitter** app. It goes without saying that you should keep these secret. If anyone was to get these keys, they could effectively access your **Twitter** account.

However, if you want to do more advanced stuff like sending tweets or deleting, you’ll need to change your access type in the "Permission" menu window. Change the access type to "Read and Write" to be able to read **Twitter** data and send tweets and select "Read, Write and access direct messages" if you want to also have access to your direct messages.

---
## Getting access to Twitter APIs using main Python tools
---

Many web services including **Twitter** require authentication, and there are many different types. 

Python library `requests` provides a pythonic way to make complex HTTP requests, and handles difficult tasks like authentication.

A common form of authentication for several web APIs is OAuth. The `requests-oauthlib` library allows `requests`'s users to easily make OAuth authenticated requests.

In [1]:
import requests
from requests_oauthlib import OAuth1

import json

# Let's define consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = 'ZFSlBaBhgAnoI5Cb5YJ8Q2iSL'
consumer_secret = 'QBFuxdiXsGypn1fEL3Nu8BQtYiFXC97RIsaCf6xv7Rq7E4Dcyg'

access_token = '16976496-hhbdqyfoVqgYbFmVz076IwWxa4vM58my0v1nJNlW0'
access_secret = '3aTWUUdNMTVWDBjxCOApzRdSiiySFM4yaZcGJ1YYaub75'

# You will authenticate yourself using OAuth1 object
auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)
print auth

In [2]:
# If the authentication was successful, you should see the name of the account print out
url_0 = 'https://api.twitter.com/1.1/account/verify_credentials.json'
res = requests.get(url_0, auth=auth)

print "My name is", res.json()["name"]


My name is Panos Ipeirotis


In [3]:
res.json()

{u'contributors_enabled': False,
 u'created_at': u'Sun Oct 26 04:16:42 +0000 2008',
 u'default_profile': False,
 u'default_profile_image': False,
 u'description': u'Professor of Data Science and Information Systems at Stern School of Business, New York University. All opinions are mine and mine alone.',
 u'entities': {u'description': {u'urls': []},
  u'url': {u'urls': [{u'display_url': u'behind-the-enemy-lines.com',
     u'expanded_url': u'http://behind-the-enemy-lines.com/',
     u'indices': [0, 22],
     u'url': u'http://t.co/ApVlO4OkZx'}]}},
 u'favourites_count': 60,
 u'follow_request_sent': False,
 u'followers_count': 3600,
 u'following': False,
 u'friends_count': 252,
 u'geo_enabled': True,
 u'has_extended_profile': True,
 u'id': 16976496,
 u'id_str': u'16976496',
 u'is_translation_enabled': False,
 u'is_translator': False,
 u'lang': u'en',
 u'listed_count': 229,
 u'location': u'New York, NY',
 u'name': u'Panos Ipeirotis',
 u'notifications': False,
 u'profile_background_color': u'

### Example 1: Read Tweets from Your Homepage

The list of all available **Twitter** API you can find here https://dev.twitter.com/rest/public. Let's use `home_timeline` that returns a collection of the most recent tweets and retweets posted by the authenticating user and the users they follow.

In [4]:
url_1 = 'https://api.twitter.com/1.1/statuses/home_timeline.json'
res = requests.get(url_1, auth=auth, params={"count": 20})

# The res object encapsulates the "response" of the server. Notice the status code that is displayed. 
# Code 200 means that things went fine
# Code 403 means that the server understood the request, but is refusing to fulfill it
# Code 404 means that the URL was not found
# Codes 5xx mean that something went wrong

print res, "Status code:", res.status_code
# Let's see how looks the url
print res.url
print "Content type:", res.headers['content-type'], '\n'

tweets = res.json()
if res.status_code == 200:
    for tweet in tweets:
        print "* ", tweet['text']
        print "----------------------------------"
else:   # You have no tweets
    pass

<Response [200]> Status code: 200
https://api.twitter.com/1.1/statuses/home_timeline.json?count=20
Content type: application/json;charset=utf-8 

*  New Stanford President Has Biotech Connection https://t.co/1OnjltrbUc
----------------------------------
*  RT @alexanderrusso: Federal Law Now Says Kids Can Walk To School Alone - FastCo via @chalkbeat https://t.co/Ax4PYdCsIh
----------------------------------
*  DNA tests solve a burning issue: Did the Explorers Club really serve mammoth in 1951? https://t.co/e796fj9AiG https://t.co/vOzyrjR65J
----------------------------------
*  Robo-Advisers are rapidly gaining marketshare vs. human financial advisors #2MA
https://t.co/Hzfq6DNWXy https://t.co/A4mBGomk29
----------------------------------
*  RT @TheEconomist: That software can now beat masters at the game of Go says a lot about where AI is going https://t.co/dDq715UjNj https://t…
----------------------------------
*  Exception handling considered important. https://t.co/Lf0gYc8xTN http

### Example 2: Search tweets by key words

**Twitter** employs a special query language.  For example, the query _"vacation?"_ will return tweets that contain the word "vacation" and are phrased as a question OR the query _"summer :)"_ will return the word "summer" with a positive attitude. [Check out more examples here](https://dev.twitter.com/rest/public/search).

We can mine tweets using either **_search_** or **_stream_**.

>The key difference between **_stream_** and **_search_** is that **_stream_** provides new data as it comes in, while **_search_** can be used to query old data. The **_search_** API is more powerful for queries, and provides faster access to a wide-range of data.

Let's search for a single tweet about "vacation", phrased as a question. Note if request phrase contains URI characters you should [encode](https://en.wikipedia.org/wiki/Percent-encoding) these queries before making the request, because these characters can play role of keys in an URL. Particularly, the question mark "?" is encoded as "%3F".

In [5]:
import urllib

query = "stern,nyu"
encoded_query = urllib.quote_plus(query)

url_2 = 'https://api.twitter.com/1.1/search/tweets.json?q=' + encoded_query
res = requests.get(url_2, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()
tweets

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?q=stern%2Cnyu


{u'search_metadata': {u'completed_in': 0.059,
  u'count': 15,
  u'max_id': 695799957400322049,
  u'max_id_str': u'695799957400322049',
  u'next_results': u'?max_id=695511186834477055&q=stern%2Cnyu&include_entities=1',
  u'query': u'stern%2Cnyu',
  u'refresh_url': u'?since_id=695799957400322049&q=stern%2Cnyu&include_entities=1',
  u'since_id': 0,
  u'since_id_str': u'0'},
 u'statuses': [{u'contributors': None,
   u'coordinates': None,
   u'created_at': u'Sat Feb 06 02:43:31 +0000 2016',
   u'entities': {u'hashtags': [],
    u'symbols': [],
    u'urls': [{u'display_url': u'fb.me/2gvJt2Yno',
      u'expanded_url': u'http://fb.me/2gvJt2Yno',
      u'indices': [75, 98],
      u'url': u'https://t.co/EEY5McLxuU'}],
    u'user_mentions': []},
   u'favorite_count': 0,
   u'favorited': False,
   u'geo': None,
   u'id': 695799957400322049,
   u'id_str': u'695799957400322049',
   u'in_reply_to_screen_name': None,
   u'in_reply_to_status_id': None,
   u'in_reply_to_status_id_str': None,
   u'in_rep

Please pay attention on how much information each tweet contains.

In [6]:
# And let's see tweets text directly
for num, tweet in enumerate(tweets['statuses']):
    print 'Tweet #{0}\t{1}\t{2}\n{3}\n'.format(num+1, tweet['created_at'], 
                                               tweet['retweet_count'], 
                                               tweet['text'].encode("utf-8"))

Tweet #1	Sat Feb 06 02:43:31 +0000 2016	0
NYU Stern | Tales in Possible | Govern Possible | EMBA Alum Richard Thomas https://t.co/EEY5McLxuU

Tweet #2	Sat Feb 06 02:30:51 +0000 2016	0
Interested in applying to NYU's #EMBA Program? Get the details now! https://t.co/TBlNqZvMCR @NYUStern https://t.co/Cs5jYZL7fh

Tweet #3	Fri Feb 05 23:32:28 +0000 2016	0
NYU Stern Interview Report: Round 1 / Adcom / On-campus https://t.co/00hqKcip01 #Education

Tweet #4	Fri Feb 05 22:03:02 +0000 2016	0
I'm clearly the most DILLIGAF panelist here #nyuswib (@ NYU Stern School of Business - @nyuniversity) https://t.co/esUAq7OLM7

Tweet #5	Fri Feb 05 21:53:16 +0000 2016	0
Check out our latest TAMID at NYU Member Profile!!! Our member of the day is Alexa Lavian, Stern '19! Learn a... https://t.co/1sxkVipffb

Tweet #6	Fri Feb 05 21:22:54 +0000 2016	0
Watch this. NYU Stern School's Scott Galloway's 2016 POV on Amazon, Google, Apple, Facebook. https://t.co/Ys58nk5bQr

Tweet #7	Fri Feb 05 20:44:54 +0000 2016	1
RT @

In [7]:
import pandas as pd
df = pd.DataFrame(tweets['statuses'])
df

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Sat Feb 06 02:43:31 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,695799957400322049,695799957400322049,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://www.facebook.com/twitter"" rel=...",NYU Stern | Tales in Possible | Govern Possibl...,False,"{u'follow_request_sent': False, u'has_extended..."
1,None,None,Sat Feb 06 02:30:51 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 15...",0,False,None,695796766801793024,695796766801793024,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",Interested in applying to NYU's #EMBA Program?...,False,"{u'follow_request_sent': False, u'has_extended..."
2,None,None,Fri Feb 05 23:32:28 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",1,False,None,695751875807875072,695751875807875072,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""https://www.socialoomph.com"" rel=""nof...",NYU Stern Interview Report: Round 1 / Adcom / ...,False,"{u'follow_request_sent': False, u'has_extended..."
3,None,"{u'type': u'Point', u'coordinates': [-73.99602...",Fri Feb 05 22:03:02 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 28...",1,False,"{u'type': u'Point', u'coordinates': [40.729085...",695729368774111232,695729368774111232,None,...,"{u'iso_language_code': u'en', u'result_type': ...","{u'full_name': u'Manhattan, NY', u'url': u'htt...",False,0,False,NaN,"<a href=""http://foursquare.com"" rel=""nofollow""...",I'm clearly the most DILLIGAF panelist here #n...,False,"{u'follow_request_sent': False, u'has_extended..."
4,None,None,Fri Feb 05 21:53:16 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,695726911457087488,695726911457087488,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://www.facebook.com/twitter"" rel=...",Check out our latest TAMID at NYU Member Profi...,False,"{u'follow_request_sent': False, u'has_extended..."
5,None,None,Fri Feb 05 21:22:54 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",0,False,None,695719270035099648,695719270035099648,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Watch this. NYU Stern School's Scott Galloway'...,False,"{u'follow_request_sent': False, u'has_extended..."
6,None,None,Fri Feb 05 20:44:54 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 27...",0,False,None,695709705096790017,695709705096790017,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,"{u'contributors': None, u'truncated': False, u...","<a href=""https://roundteam.co"" rel=""nofollow"">...",RT @NYUSternRisk: The abnormal #globaleconomy ...,False,"{u'follow_request_sent': False, u'has_extended..."
7,None,None,Fri Feb 05 20:42:35 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 19...",0,False,None,695709124529623041,695709124529623041,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,1,False,NaN,"<a href=""http://www.hootsuite.com"" rel=""nofoll...",The abnormal #globaleconomy according to Profe...,False,"{u'follow_request_sent': False, u'has_extended..."
8,None,None,Fri Feb 05 20:29:13 +0000 2016,"{u'symbols': [], u'user_mentions': [], u'hasht...",2,False,None,695705760949473280,695705760949473280,None,...,"{u'iso_language_code': u'en', u'result_type': ...",None,False,0,False,NaN,"<a href=""http://ifttt.com"" rel=""nofollow"">IFTT...",Stern Building Turnstiles to Require American ...,False,"{u'follow_request_sent': False, u'has_extended..."
9,None,None,Fri Feb 05 18:12:39 +0000 2016,"{u'symbols': 

In [8]:
# If we want to flatten the nested JSON entries
# we can use the json_normalize function
# http://pandas-docs.github.io/pandas-docs-travis/io.html#normalization
import pandas as pd
from pandas.io.json import json_normalize
data = json_normalize(tweets['statuses'])
df_flat = pd.DataFrame(data)
df_flat

,contributors,coordinates,coordinates.coordinates,coordinates.type,created_at,entities.hashtags,entities.media,entities.symbols,entities.urls,entities.user_mentions,...,user.profile_sidebar_fill_color,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.url,user.utc_offset,user.verified
0,None,NaN,NaN,NaN,Sat Feb 06 02:43:31 +0000 2016,[],NaN,[],"[{u'url': u'https://t.co/EEY5McLxuU', u'indice...",[],...,7AC3EE,3D1957,True,False,KexiKeepItMovin,5227,Quito,None,-18000,False
1,None,NaN,NaN,NaN,Sat Feb 06 02:30:51 +0000 2016,"[{u'indices': [32, 37], u'text': u'EMBA'}]",[{u'expanded_url': u'http://twitter.com/IvyExe...,[],"[{u'url': u'https://t.co/TBlNqZvMCR', u'indice...","[{u'indices': [92, 101], u'id_str': u'15837820...",...,020D01,61941F,True,False,IvyExec,14078,Eastern Time (US & Canada),http://t.co/BI8X8Iw580,-18000,False
2,None,NaN,NaN,NaN,Fri Feb 05 23:32:28 +0000 2016,"[{u'indices': [80, 90], u'text': u'Education'}]",NaN,[],"[{u'url': u'https://t.co/00hqKcip01', u'indice...",[],...,DDEEF6,333333,True,False,EducationMavin1,43612,None,http://t.co/4Hvksepoeb,NaN,False
3,None,NaN,"[-73.9960221, 40.72908585]",Point,Fri Feb 05 22:03:02 +0000 2016,"[{u'indices': [44, 52], u'text': u'nyuswib'}]",NaN,[],"[{u'url': u'https://t.co/esUAq7OLM7', u'indice...","[{u'indices': [87, 100], u'id_str': u'28421825...",...,F3F4F1,153444,False,False,AlmostMedia,8235,Eastern Time (US & Canada),None,-18000,False
4,None,NaN,NaN,NaN,Fri Feb 05 21:53:16 +0000 2016,[],NaN,[],"[{u'url': u'https://t.co/1sxkVipffb', u'indice...",[],...,000000,000000,False,False,nyutamid,222,Atlantic Time (Canada),http://t.co/UETrjm2SVP,-14400,False
5,None,NaN,NaN,NaN,Fri Feb 05 21:22:54 +0000 2016,[],NaN,[],"[{u'url': u'https://t.co/Ys58nk5bQr', u'indice...",[],...,EFEFEF,333333,True,False,tbrunelle,11566,Central Time (US & Canada),https://t.co/td34ezAagj,-21600,False
6,None,NaN,NaN,NaN,Fri Feb 05 20:44:54 +0000 2016,"[{u'indices': [31, 45], u'text': u'globalecono...",NaN,[],"[{u'url': u'https://t.co/Jv4pj99Fzk', u'indice...","[{u'indices': [3, 16], u'id_str': u'277118349'...",...,DDEEF6,333333,True,False,TrueWeRISK,6089,Europe/Rome,https://t.co/pY15bvGUJS,3600,False
7,None,NaN,NaN,NaN,Fri Feb 05 20:42:35 +0000 2016,"[{u'indices': [13, 27], u'text': u'globalecono...",NaN,[],"[{u'url': u'https://t.co/Jv4pj99Fzk', u'indice...","[{u'indices': [51, 59], u'id_str': u'19224439'...",...,C4A2EB,3D1957,False,False,NYUSternRisk,734,Quito,http://t.co/9AfUOPZQoN,-18000,False
8,None,NaN,NaN,NaN,Fri Feb 05 20:29:13 +0000 2016,[],NaN,[],"[{u'url': u'https://t.co/yV3ogvsjo2', u'indice...",[],...,DDEEF6,333333,True,False,WSLocal,1052,Eastern Time (US & Canada),http://t.co/23FArEqwqI,-18000,False
9,None,NaN,NaN,NaN,Fri Feb 05 18:12:39 +0000 2016,[],NaN,[],"[{u'url': u'https://t.co/TzGPNYQHN5', u'indice...","[{u'indices': [0, 15], u'id_str': u'152530023'...",...,DDEEF6,333333,True,False,mallyvai,12233,Pacific Time (US & Canada),https://t.co/AMy2O2KKqR,-28800,False


In [9]:
mylist = [
    {"FullName": {
            "Given": {
                "First": "Panos", 
                "Middle": "Gregory"
            }, 
            "Family": "Ipeirotis"
        }, 
     "Age": 40
    },
    {"FullName": {
            "Given": {
                "First": "John", 
                "Middle": "Reuben"
            }, 
            "Family": "Smith"
        }, 
     "Age": 25
    },
    {"FullName": {
            "Given": {
                "First": "Jane", 
                "Middle": "Janet"
            }, 
            "Family": "Smith"}, 
     "Age": 22
    }
]

df_test = pd.DataFrame(mylist)
df_test

,Age,FullName
0,40,"{u'Given': {u'Middle': u'Gregory', u'First': u..."
1,25,"{u'Given': {u'Middle': u'Reuben', u'First': u'..."
2,22,"{u'Given': {u'Middle': u'Janet', u'First': u'J..."


In [10]:
from pandas.io.json import json_normalize
df_test_normalized = json_normalize(mylist)
df_test_normalized

,Age,FullName.Family,FullName.Given.First,FullName.Given.Middle
0,40,Ipeirotis,Panos,Gregory
1,25,Smith,John,Reuben
2,22,Smith,Jane,Janet


You may create very specialized and more concrete queries. 

Let's find 5 tweets that contains the word "python" or "IPython" near New York. We can provide this as a `geocode` with a lattitude, longitude and radius. We can also specify time range of tweets appearance (with the help of `since` and `until` key words) and the tweet language (the `lang` parameter restricts tweets to the given language). Additional parameters allows build very complicated search requests.

In [11]:
url_2_2 = 'https://api.twitter.com/1.1/search/tweets.json'

params = {
    "count": 10, 
    "geocode": '40.7127,-74.0059,15mi',
    "lang": 'en',
    "q": 'python'
}

res = requests.get(url_2_2, auth=auth, params=params)

print res, res.status_code, res.headers['content-type']
print res.url

tweets = res.json()

for num, tweet in enumerate(tweets['statuses']):
    print "-----------------------------"
    print 'Tweet #{0}\t{1}\n{2}\n'.format(num+1, tweet['created_at'], tweet['text'].encode("utf-8"))

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/search/tweets.json?count=10&lang=en&geocode=40.7127%2C-74.0059%2C15mi&q=python
-----------------------------
Tweet #1	Thu Feb 04 14:32:20 +0000 2016
Can you recommend anyone for this #job? Python Developer - https://t.co/PHP0SGZKLZ #NettempsJobs #NewYork, NY #IT #Hiring #CareerArc

-----------------------------
Tweet #2	Wed Feb 03 23:27:36 +0000 2016
What I do for the kids...I held a freakin python!!! @ Holy Family School https://t.co/tgTTuN8rLo

-----------------------------
Tweet #3	Wed Feb 03 19:40:39 +0000 2016
Harrison Reed Computer Consulting: Shell Scripting, Python, Perl, Trading, SQL (#NewYork, NY) https://t.co/8LCKFmUoeJ #InvestmentBanking

-----------------------------
Tweet #4	Wed Feb 03 15:54:40 +0000 2016
#waybackwednesday to the first matte black Python @bracaletti Gaia clutch a year and a half ago… https://t.co/aUmENlSw7l

-----------------------------
Tweet #5	Wed Feb 03 02:27:26 +0000 2016

### Example 3: Get a list of all your followers

**Twitter** GET request `followers/list` returns a cursored collection of user objects for users following the specified user. At this time, results are ordered with the most recent following first.

In [12]:
url_3 = 'https://api.twitter.com/1.1/followers/list.json'
res = requests.get(url_3, auth=auth)
print res, res.status_code, res.headers['content-type']
print res.url

followers = res.json()
followers

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/followers/list.json


{u'next_cursor': 1524023813801389917,
 u'next_cursor_str': u'1524023813801389917',
 u'previous_cursor': 0,
 u'previous_cursor_str': u'0',
 u'users': [{u'blocked_by': False,
   u'blocking': False,
   u'contributors_enabled': False,
   u'created_at': u'Wed Aug 24 08:56:35 +0000 2011',
   u'default_profile': True,
   u'default_profile_image': False,
   u'description': u'20. New York University. Tech and Entrepreneurship',
   u'entities': {u'description': {u'urls': []}},
   u'favourites_count': 122,
   u'follow_request_sent': False,
   u'followers_count': 104,
   u'following': False,
   u'friends_count': 179,
   u'geo_enabled': False,
   u'has_extended_profile': False,
   u'id': 361126970,
   u'id_str': u'361126970',
   u'is_translation_enabled': False,
   u'is_translator': False,
   u'lang': u'en',
   u'listed_count': 3,
   u'location': u'New York, NY',
   u'muting': False,
   u'name': u'Karan Magu',
   u'notifications': False,
   u'profile_background_color': u'C0DEED',
   u'profile_backg

In [13]:
# And let's see only followers' name
for num, follower in enumerate(followers['users']):
    print 'My follower #{0}\t{1}'.format(num+1, follower['name'].encode("utf-8"))
    

My follower #1	Karan Magu
My follower #2	Dr. Eva
My follower #3	Jérôme Gleizes
My follower #4	than
My follower #5	kuhan_mti
My follower #6	Stefan Vetter
My follower #7	Somisetty
My follower #8	DTAC
My follower #9	Yongsung Kim
My follower #10	UW AI Research
My follower #11	Annalyn Ng
My follower #12	Danielle Szafir
My follower #13	Matthew Voshell
My follower #14	Shengli Hu
My follower #15	Yuanyang Liu
My follower #16	luiggino
My follower #17	Paris Lambert
My follower #18	Michael Feldman
My follower #19	TheVideoCorporation
My follower #20	Brennan


In [14]:
# And now an example of using a "cursor" to go through pages of results
cursor = res.json().get("next_cursor_str")
res = requests.get(url_3, auth=auth, params={"cursor": cursor})
print res, res.status_code, res.headers['content-type']
print res.url

followers = res.json()
for num, follower in enumerate(followers['users']):
    print 'My follower #{0}\t{1}'.format(num+1, follower['name'].encode("utf-8"))
    


<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/followers/list.json?cursor=1524023813801389917
My follower #1	Kevin Boudreau
My follower #2	ZarMoney App
My follower #3	Andrey Seleznov
My follower #4	Nicole Mardis
My follower #5	David A Isaacs
My follower #6	MTurk Crowd
My follower #7	Business Unbound
My follower #8	Ian Dennis Miller
My follower #9	Kate Dericott
My follower #10	srinivasu
My follower #11	Daemo
My follower #12	fehmi tuncer
My follower #13	Don Turnbull
My follower #14	Nikolaos Karianakis
My follower #15	Matthew Gombolay
My follower #16	Rajeev Jaiswal
My follower #17	Rahul
My follower #18	nedoconductor
My follower #19	Sebastian
My follower #20	Derrick


### Example 4:  Finding what is trending

According to the [tweepy API](http://tweepy.readthedocs.org/en/v3.5.0/api.html), we can return the top 50 trending topics for a specific location, where the location is a `WOEID (Yahoo Where on Earth ID)`. 

The `WOEID` is a unique identifier, similar to zipcodes, but that expand worldwide. For example, New York has a `WOEID` of 2459115. You can search for `WOEID`'s here: http://woeid.rosselliot.co.nz/.

Let's see the top trending topics in New York

In [15]:
url_4 = 'https://api.twitter.com/1.1/trends/place.json?id=2459115'
res = requests.get(url_4, auth=auth)

print res, res.status_code, res.headers['content-type']
print res.url

top50_trends = res.json()
top50_trends

<Response [200]> 200 application/json;charset=utf-8
https://api.twitter.com/1.1/trends/place.json?id=2459115


[{u'as_of': u'2016-02-06T05:25:11Z',
  u'created_at': u'2016-02-06T05:22:44Z',
  u'locations': [{u'name': u'New York', u'woeid': 2459115}],
  u'trends': [{u'name': u'#ImageAwards',
    u'promoted_content': None,
    u'query': u'%23ImageAwards',
    u'tweet_volume': 38466,
    u'url': u'http://twitter.com/search?q=%23ImageAwards'},
   {u'name': u'#EVOL',
    u'promoted_content': None,
    u'query': u'%23EVOL',
    u'tweet_volume': 58073,
    u'url': u'http://twitter.com/search?q=%23EVOL'},
   {u'name': u'#USAvCAN',
    u'promoted_content': None,
    u'query': u'%23USAvCAN',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23USAvCAN'},
   {u'name': u'#RIPTwitter',
    u'promoted_content': None,
    u'query': u'%23RIPTwitter',
    u'tweet_volume': None,
    u'url': u'http://twitter.com/search?q=%23RIPTwitter'},
   {u'name': u'#AddAdjectivesToFilmTitles',
    u'promoted_content': None,
    u'query': u'%23AddAdjectivesToFilmTitles',
    u'tweet_volume': 11436,
    u'url

In [16]:
# And let's see only the first 10 trends names and the respective URL
for i, trend in enumerate(top50_trends[0]['trends'][:10]):
    print '{0} - {1} - URL: {2}'.format(i+1, trend['name'], trend['url'])

1 - #ImageAwards - URL: http://twitter.com/search?q=%23ImageAwards
2 - #EVOL - URL: http://twitter.com/search?q=%23EVOL
3 - #USAvCAN - URL: http://twitter.com/search?q=%23USAvCAN
4 - #RIPTwitter - URL: http://twitter.com/search?q=%23RIPTwitter
5 - #AddAdjectivesToFilmTitles - URL: http://twitter.com/search?q=%23AddAdjectivesToFilmTitles
6 - Bulls - URL: http://twitter.com/search?q=Bulls
7 - Mavs - URL: http://twitter.com/search?q=Mavs
8 - Avery Bradley - URL: http://twitter.com/search?q=%22Avery+Bradley%22
9 - Jimmy Butler - URL: http://twitter.com/search?q=%22Jimmy+Butler%22
10 - Jermaine Jones - URL: http://twitter.com/search?q=%22Jermaine+Jones%22


### Example 5: Streaming

**Twitter** offers a [Streaming API](https://dev.twitter.com/streaming/overview) to make it easier to query streams of tweets.  The Stream API encapsulates some pain points of REST access to ensure that Stream calls don't exceed the rate limit. These tool allows to get tweet data when it appears in real time. There are three stream types:

- `Public Streams:` Streams of public data flowthing through **Twitter**. Suitable for followign specific users, topics or for data mining.
- `User Streams:` Single-user streams. Containing roughly all of the data corresponding with a single user's view of **Twitter**. 
- `Site Streams:` The multi-user version of user streams.

>Connecting to the streaming API requires keeping a persistent HTTP connection open. In many cases this involves thinking about your application differently than if you were interacting with the REST API. An app which connects to the Streaming APIs will not be able to establish a connection in response to a user request. Instead, the code for maintaining the Streaming connection is typically run in a process separate from the process which handles HTTP requests. The streaming process gets the input tweets and performs any parsing, filtering, and/or aggregation needed before storing the result to a data store. The HTTP handling process queries the data store for results in response to user requests. While this model is more complex than the first example, the benefits from having a realtime stream of tweet data make the integration worthwhile for many types of apps.

With `requests.Response.iter_lines()` you can easily iterate over streaming APIs including the **Twitter Streaming API**. Simply set `stream` parameter to `True` and iterate over the response with `iter_lines()`. 

Depending on the search term/terms, we can get tons of tweets within a few minutes.

A working example that gathers all the new tweets with the _#iphone_ hashtag:

In [23]:
import time
import urllib

query = "#iphone"
encoded_query = urllib.quote_plus(query)
url_5 = 'https://stream.twitter.com/1.1/statuses/filter.json?track=' + encoded_query
res = requests.get(url_5, auth=auth, stream=True)

print res, res.status_code, res.headers['content-type']
print res.url, "\n"

# Let's measure the time elapsed after the start of streaming 
start = time.time()
stop_after = 10
tweets_printed = 0
for line in res.iter_lines():
    # filter out keep-alive new lines
    if line:
        parsed_line = json.loads(line)
        print "Elapsed:", time.time()-start, "sec",  parsed_line["created_at"], "\n", parsed_line['text'].encode("utf-8"), "\n"
        tweets_printed = tweets_printed + 1
        if tweets_printed>stop_after:
            break

print "Time Elapsed for getting", stop_after, " tweets about ", query, " ==>" , time.time()-start

<Response [200]> 200 application/json
https://stream.twitter.com/1.1/statuses/filter.json?track=%23iphone 

Elapsed: 1.04557609558 sec Sat Feb 06 05:30:48 +0000 2016 
THE Gadget Sleeve Shop | https://t.co/CrDbc3VZNW | #HTConeM8 #SurfacePro3 #GalaxS5 #iPhone #Nexus #Motorola #Lumia #Kindle 

Elapsed: 1.77153611183 sec Sat Feb 06 05:30:49 +0000 2016 
I've finished the Dusty Dungeon quest! Are you able to fulfil the same task? https://t.co/zbugXWpgN1 #iphone,#iphonegames,#gameinsight 

Elapsed: 3.25731992722 sec Sat Feb 06 05:30:51 +0000 2016 
iPhone Organic Leather Sleeve MR CARRAWAY | https://t.co/6fOrmCb9TC | #iphone #iphone5 #iPhone5Sleeve #iPhoneSleeve #iPhone5sSleeve #iPho… 

Elapsed: 4.16797804832 sec Sat Feb 06 05:30:51 +0000 2016 
iPhone Organic Leather Sleeve | PAPAYA | https://t.co/QOU6XwEHis | #iphone #iphone5 #iPhone5Sleeve #iPhoneSleeve #iPhone5sSleeve #iPhone5s 

Elapsed: 4.3902220726 sec Sat Feb 06 05:30:52 +0000 2016 
iPhone Deerskin Leather Sleeve |PURPLE LEAF https://t.

---
## Exercises
---

## Exercise #1: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library display, the list of all your friends and sort them by its followers amounts in descending order. 

#### HINT: `GET friends/list`  will be helpfull for you here.

## Exercise #2: 

> Using [Twitter API](https://dev.twitter.com/rest/public) and `requests` Python library, display the list of 5 (or less if there is not so much) most recent tweets liked by you, which were also retweeted (`retweet_count` is not equal to zero). You shoud show the date of creation, author of the tweet and its text.

#### HINT: `GET favorites/list` will be helpfull for you here

## Exercise #3: 

> Using [Twitter Streaming API](https://dev.twitter.com/streaming/overview) and `requests` Python library count how many tweets with hashtags _#twitter_, _#tweet_ and _#world_ are appearing each minute over 5 minutes and display these 5 numbers. Please display also the shortest and the longest time period between to closest tweets for all 5 measures.

#### HINT: `POST statuses/filter` will be helpfull for you here